#### This analysis follows the steps from `data_loading_and_cleaning` and `Exploratory_data_analytics`, so please review those sections before proceeding with this notebook.

In [1]:
import os
import sys 
import pandas as pd
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../data'))

In [2]:
import eda as e
import ab_testing as ab

In [3]:
df = e.load_data('../data/transformed_data.csv')

c:\Users\Hp\Desktop\Kifiya\Week3\ACIS-Claim-Analytics\src\eda.py:6: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [4]:
pd.set_option('display.max_columns', None)
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,CustomValueEstimate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,CrossBorder,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,Year,Month
0,145249,12827,2015-03-01,True,ZA,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,No,No,No,No,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,2015,3
1,145249,12827,2015-05-01,True,ZA,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,No,No,No,No,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,2015,5
2,145249,12827,2015-07-01,True,ZA,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,No,No,No,No,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,2015,7
3,145255,12827,2015-05-01,True,ZA,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,No,No,No,No,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,2015,5
4,145255,12827,2015-07-01,True,ZA,Close Corporation,Mr,English,First National Bank,Current account,Not specified,Male,South Africa,Gauteng,1459,Rand East,Rand East,Mobility - Motor,44069150.0,Passenger Vehicle,2004,MERCEDES-BENZ,E 240,6.0,2597.0,130.0,S/D,4.0,6/2002,119300.0,Yes,No,119300,More than 6 months,No,No,No,No,119300.00,Monthly,584.6468,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,2015,7


In [17]:
df['Title'].value_counts()

Title
Mr      933003
Mrs      45850
Ms       13269
Miss      6614
Dr         810
Name: count, dtype: int64

## A/B Testing

In [9]:
# select metrics 
metrics_df = ab.select_metrics(df)

In [33]:
# Hypothesis 1: Risk differences across provinces (Kruskal-Wallis Test)
h_stat, p_value = ab.perform_kruskal_test(df, 'Province', 'TotalClaims')
print(f"Risk differences across provinces (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

Risk differences across provinces (Kruskal-Wallis): h_stat = 110.79079324392988, p_value = 2.6186589474507583e-20
Reject the null hypothesis


In [32]:
# Hypothesis 2: Margin (profit) differences across provinces (Kruskal-Wallis Test)
h_stat, p_value = ab.perform_kruskal_test(df, 'Province', 'TotalPremium')
print(f"Risk differences across provinces (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

Risk differences across provinces (Kruskal-Wallis): h_stat = 4621.458238736816, p_value = 0.0
Reject the null hypothesis


In [38]:
# Hypothesis 3: Risk differences between vehicle types (Z Test and Kruskal-Wallis Test)
group_a_df, group_b_df = ab.data_segmentation(df, 'VehicleType', 'Passenger Vehicle', 'Medium Commercial')
z_stat, p_value = ab.perform_z_test(group_a_df, group_b_df, 'TotalPremium')
print(f"Margin differences between vehicle types: z_stat = {z_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

h_stat, p_value = ab.perform_kruskal_test(df, 'VehicleType', 'TotalClaims')
print(f"Risk differences between vehicle types (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

Margin differences between vehicle types: z_stat = -14.821543633660712, p_value = 0.0
Reject the null hypothesis
Risk differences between vehicle types (Kruskal-Wallis): h_stat = 1.6007195280102569, p_value = 0.8086628076893109
Fail to reject the null hypothesis


In [35]:
# Hypothesis 4: Margin (profit) differences between vehicle types (T-Test and Wilcoxon Rank-Sum Test)
group_a_df, group_b_df = ab.data_segmentation(df, 'VehicleType', 'Passenger Vehicle', 'Medium Commercial')

t_stat, p_value = ab.perform_t_test(group_a_df, group_b_df, 'TotalPremium')
print(f"Margin differences between vehicle types (T-Test): t_stat = {t_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

w_stat, p_value = ab.perform_wilcoxon_test(group_a_df, group_b_df, 'TotalPremium')
print(f"Margin differences between vehicle types (Wilcoxon Rank-Sum): w_stat = {w_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

Margin differences between vehicle types (T-Test): t_stat = -14.821543633660712, p_value = 1.3084367425095875e-49
Reject the null hypothesis
Margin differences between vehicle types (Wilcoxon Rank-Sum): w_stat = -11.384280512412468, p_value = 5.0079871574257575e-30
Reject the null hypothesis


In [36]:
# Hypothesis 5: Risk differences between Women and Men (Wilcoxon Rank-Sum Test)
group_a_df, group_b_df = ab.data_segmentation(df, 'Gender', 'Female', 'Male')

w_stat, p_value = ab.perform_wilcoxon_test(group_a_df, group_b_df, 'TotalClaims')
print(f"Risk differences between Women and Men (Wilcoxon Rank-Sum): w_stat = {w_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

Risk differences between Women and Men (Wilcoxon Rank-Sum): w_stat = -0.37172076447581864, p_value = 0.7101007632747667
Fail to reject the null hypothesis


In [37]:
# Hypothesis 6: Risk differences between Titles (Mr vs Mrs) (ANOVA and Kruskal-Wallis Test)
df_titles = df[df['Title'].isin(['Mr', 'Mrs'])]

# One-way ANOVA
f_stat, p_value = ab.perform_anova_test(df_titles, 'Title', 'TotalClaims')
print(f"Risk differences between Titles (Mr vs Mrs) (ANOVA): f_stat = {f_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

# Kruskal-Wallis Test
h_stat, p_value = ab.perform_kruskal_test(df_titles, 'Title', 'TotalClaims')
print(f"Risk differences between Titles (Mr vs Mrs) (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

# If you want to include more titles, you can adjust the list in the 'isin' method
df_titles_extended = df[df['Title'].isin(['Mr', 'Mrs', 'Ms', 'Miss', 'Dr'])]

# One-way ANOVA for extended titles
f_stat, p_value = ab.perform_anova_test(df_titles_extended, 'Title', 'TotalClaims')
print(f"Risk differences between extended Titles (ANOVA): f_stat = {f_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

# Kruskal-Wallis Test for extended titles
h_stat, p_value = ab.perform_kruskal_test(df_titles_extended, 'Title', 'TotalClaims')
print(f"Risk differences between extended Titles (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

Risk differences between Titles (Mr vs Mrs) (ANOVA): f_stat = 3.4123173651766883, p_value = 0.0647118132494381
Fail to reject the null hypothesis
Risk differences between Titles (Mr vs Mrs) (Kruskal-Wallis): h_stat = 13.592378029398487, p_value = 0.000227105751050255
Reject the null hypothesis
Risk differences between extended Titles (ANOVA): f_stat = 1.1687415400864312, p_value = 0.32230275744476866
Fail to reject the null hypothesis
Risk differences between extended Titles (Kruskal-Wallis): h_stat = 18.927768373167964, p_value = 0.0008120452768202463
Reject the null hypothesis


### The Results Indicate Some Interesting Findings:

#### Risk Differences Between Titles (Mr vs Mrs):
- **ANOVA**: ( f_stat = 3.412 ), ( p_value = 0.065 )
  - **Conclusion**: Fail to reject the null hypothesis. This suggests no significant difference in risk (TotalClaims) between “Mr” and “Mrs” based on ANOVA.
- **Kruskal-Wallis**: ( h_stat = 13.592 ), ( p_value = 0.0002 )
  - **Conclusion**: Reject the null hypothesis. This suggests a significant difference in risk (TotalClaims) between “Mr” and “Mrs” based on the Kruskal-Wallis test.

#### Risk Differences Between Extended Titles:
- **ANOVA**: ( f_stat = 1.169 ), ( p_value = 0.322 )
  - **Conclusion**: Fail to reject the null hypothesis. This suggests no significant difference in risk (TotalClaims) among the extended titles based on ANOVA.
- **Kruskal-Wallis**: ( h_stat = 18.928 ), ( p_value = 0.0008 )
  - **Conclusion**: Reject the null hypothesis. This suggests a significant difference in risk (TotalClaims) among the extended titles based on the Kruskal-Wallis test.

### Interpretation

- **ANOVA vs. Kruskal-Wallis**: The discrepancy between the ANOVA and Kruskal-Wallis results could be due to the assumptions each test makes. ANOVA assumes normally distributed data and equal variances, while Kruskal-Wallis is a non-parametric test that does not make these assumptions. The significant results from the Kruskal-Wallis test suggest that there might be differences in the distributions of TotalClaims across titles that ANOVA did not detect due to its assumptions.
- **Imbalance in Data**: The large number of “Mr” entries compared to other titles might influence the results. The Kruskal-Wallis test is more robust to such imbalances, which might explain why it detected significant differences.

### Next Steps

- **Further Analysis**: Consider visualizing the data distributions (e.g., box plots) to better understand the differences in TotalClaims across titles.
- **Data Balancing**: If feasible, balance the data (e.g., through resampling) to see if the results hold.
- **Additional Tests**: Explore other non-parametric tests or regression models to further validate the findings.


In [20]:
# Additional factors to explore
additional_factors = ['MaritalStatus', 'Gender', 'Province', 'PostalCode', 'RegistrationYear', 
                      'NewVehicle', 'cubiccapacity', 'kilowatts', 'NumberOfDoors']

In [23]:
# Function to perform ANOVA and Kruskal-Wallis tests for additional factors
def test_additional_factors(df, factors, target):
    for factor in factors:
        print(f"Testing {factor} against {target}")
        
        # ANOVA Test
        f_stat, p_value = ab.perform_anova_test(df, factor, target)
        print(f"ANOVA: f_stat = {f_stat}, p_value = {p_value}")
        print(ab.analyze_results(p_value))
        
        # Kruskal-Wallis Test
        h_stat, p_value = ab.perform_kruskal_test(df, factor, target)
        print(f"Kruskal-Wallis: h_stat = {h_stat}, p_value = {p_value}")
        print(ab.analyze_results(p_value))

In [24]:
# Test additional factors against TotalClaims
test_additional_factors(df, additional_factors, 'TotalClaims')

Testing MaritalStatus against TotalClaims
ANOVA: f_stat = 0.7757878668605783, p_value = 0.4603412278413175
Fail to reject the null hypothesis
Kruskal-Wallis: h_stat = 6.408367703849895, p_value = 0.040592017219140755
Reject the null hypothesis
Testing Gender against TotalClaims
ANOVA: f_stat = 2.2756020200642526, p_value = 0.1027355735138275
Fail to reject the null hypothesis
Kruskal-Wallis: h_stat = 16.63281801811604, p_value = 0.0002444721872598982
Reject the null hypothesis
Testing Province against TotalClaims
ANOVA: f_stat = 6.0534220286843405, p_value = 8.188078379290596e-08
Reject the null hypothesis
Kruskal-Wallis: h_stat = 110.79079324392988, p_value = 2.6186589474507583e-20
Reject the null hypothesis
Testing PostalCode against TotalClaims
ANOVA: f_stat = 0.9409358563539465, p_value = 0.8948604188712437
Fail to reject the null hypothesis
Kruskal-Wallis: h_stat = 1452.9129351537897, p_value = 4.2794525807755786e-30
Reject the null hypothesis
Testing RegistrationYear against Tota

### Select Test Method

Based on the dataset I have, I would recommend using the Kruskal-Wallis test. Here's why:

- **Non-normality**: The dataset contains categorical variables and count data, which are unlikely to be normally distributed. Kruskal-Wallis is a non-parametric test that doesn't assume normality.
- **Unequal variances**: The dataset has a large number of observations, and it's likely that the variances of the groups are not equal. Kruskal-Wallis doesn't assume equal variances.
- **Comparing medians**: Since the dataset contains count data and categorical variables, comparing medians might be more meaningful than comparing means. Kruskal-Wallis compares medians.
- **Robustness**: Kruskal-Wallis is a robust test that can handle outliers and non-normal data, which is common in real-world datasets.

Some potential research questions that I could explore using the Kruskal-Wallis test include:

- Is there a significant difference in the distribution of vehicle types (e.g., passenger vehicle, medium commercial, heavy commercial) across different regions (e.g., Transvaal South East, Natal 1, East London)?
- Is there a significant difference in the distribution of cover types (e.g., own damage, passenger liability, windscreen) across different vehicle makes (e.g., Toyota, Mercedes-Benz, Volkswagen)?
- Is there a significant difference in the distribution of policy values (e.g., total premium, total claims) across different cover groups (e.g., comprehensive, third party, fire and theft)?

By using the Kruskal-Wallis test, I can determine if there are significant differences in the medians of these variables across different groups, without assuming normality or equal variances.

So, I would recommend using the Kruskal-Wallis test to analyze the dataset.

### Result of Testing Additional factors against TotalClaims

I selected the Kruskal-Wallis test. Based on the results of the Kruskal-Wallis test, here are some recommendations:

#### **Reject the Null Hypothesis**

- **Province against TotalClaims**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 110.79079324392988, p_value = 2.6186589474507583e-20), indicating that there is a significant difference in TotalClaims across different provinces.
- **cubiccapacity against TotalClaims**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 241.87492325511215, p_value = 4.4404208730337115e-10), indicating that there is a significant difference in TotalClaims across different cubic capacities.
- **kilowatts against TotalClaims**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 173.77483816512094, p_value = 1.0024031467947507e-08), indicating that there is a significant difference in TotalClaims across different kilowatts.
- **PostalCode against TotalClaims**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 1452.9129351537897, p_value = 4.2794525807755786e-30), indicating that there is a significant difference in TotalClaims across different postal codes.
- **RegistrationYear against TotalClaims**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 81.0943577247853, p_value = 4.068523477133671e-08), indicating that there is a significant difference in TotalClaims across different registration years.
- **NewVehicle against TotalClaims**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 4.3819164797624195, p_value = 0.03632213569611937), indicating that there is a significant difference in TotalClaims across different new vehicle categories.
- **Gender against TotalClaims**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 16.63281801811604, p_value = 0.000244472187259898), indicating that there is a significant difference in TotalClaims across different genders.

### Recommendations

- **Investigate the relationship between Province and TotalClaims** further, as there appears to be a significant difference in TotalClaims across different provinces.
- **Explore the relationship between cubiccapacity, kilowatts, and TotalClaims**, as there appears to be a significant difference in TotalClaims across different cubic capacities and kilowatts.
- **Investigate the relationship between PostalCode and TotalClaims**, as there appears to be a significant difference in TotalClaims across different postal codes.
- **Investigate the relationship between RegistrationYear and TotalClaims**, as there appears to be a significant difference in TotalClaims across different registration years.
- **Investigate the relationship between NewVehicle and TotalClaims**, as there appears to be a significant difference in TotalClaims across different new vehicle categories.
- **Investigate the relationship between Gender and TotalClaims**, as there appears to be a significant difference in TotalClaims across different genders.

In [29]:
# Test 1: Vehicle types across regions
h_stat, p_value = ab.perform_kruskal_test(df, 'MainCrestaZone', 'VehicleType')
print(f"Vehicle types across regions (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

# Test 2: Cover types across vehicle makes
h_stat, p_value = ab.perform_kruskal_test(df, 'make', 'CoverType')
print(f"Cover types across vehicle makes (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

# Test 3: Total premium across cover groups
h_stat, p_value = ab.perform_kruskal_test(df, 'CoverGroup', 'TotalPremium')
print(f"Total premium across cover groups (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

# Test 4: Total claims across cover groups
h_stat, p_value = ab.perform_kruskal_test(df, 'CoverGroup', 'TotalClaims')
print(f"Total claims across cover groups (Kruskal-Wallis): h_stat = {h_stat}, p_value = {p_value}")
print(ab.analyze_results(p_value))

Vehicle types across regions (Kruskal-Wallis): h_stat = 8182.31225093489, p_value = 0.0
Reject the null hypothesis
Cover types across vehicle makes (Kruskal-Wallis): h_stat = 1110.9234102457926, p_value = 2.6069560488147075e-203
Reject the null hypothesis
Total premium across cover groups (Kruskal-Wallis): h_stat = 20239.30892667186, p_value = 0.0
Reject the null hypothesis
Total claims across cover groups (Kruskal-Wallis): h_stat = 795.9418492010841, p_value = 1.0219151852073095e-161
Reject the null hypothesis


### Based on the Results of the Kruskal-Wallis Test, Here Are Some Recommendations:

#### **Reject the Null Hypothesis**

- **Vehicle types across regions**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 8182.31225093489, p_value = 0.0), indicating that there is a significant difference in vehicle types across different regions.
- **Cover types across vehicle makes**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 1110.9234102457926, p_value = 2.6069560488147075e-203), indicating that there is a significant difference in cover types across different vehicle makes.
- **Total premium across cover groups**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 20239.30892667186, p_value = 0.0), indicating that there is a significant difference in total premium across different cover groups.
- **Total claims across cover groups**: The Kruskal-Wallis test rejects the null hypothesis (h_stat = 795.9418492010841, p_value = 1.0219151852073095e-161), indicating that there is a significant difference in total claims across different cover groups.

### Recommendations

- **Investigate the relationship between vehicle types and regions** further, as there appears to be a significant difference in vehicle types across different regions.
- **Explore the relationship between cover types and vehicle makes**, as there appears to be a significant difference in cover types across different vehicle makes.
- **Investigate the relationship between total premium and cover groups**, as there appears to be a significant difference in total premium across different cover groups.
- **Investigate the relationship between total claims and cover groups**, as there appears to be a significant difference in total claims across different cover groups.

### Potential Next Steps

- **Conduct a post-hoc analysis** to determine which specific regions have significantly different vehicle types.
- **Conduct a post-hoc analysis** to determine which specific vehicle makes have significantly different cover types.
- **Conduct a post-hoc analysis** to determine which specific cover groups have significantly different total premiums.
- **Conduct a post-hoc analysis** to determine which specific cover groups have significantly different total claims.

### Business Implications

- The results suggest that there may be **regional differences in vehicle types**, which could inform marketing and sales strategies.
- The results suggest that there may be **differences in cover types across vehicle makes**, which could inform product development and pricing strategies.
- The results suggest that there may be **differences in total premium across cover groups**, which could inform pricing and revenue management strategies.
- The results suggest that there may be **differences in total claims across cover groups**, which could inform risk management and claims processing strategies.